# Training a Neural Net on classification tasks with different (probably biased) Word Embeddings

In [ ]:
!pip install git+https://github.com/millawell/bias-ml-dh.git#subdirectory=material/notebooks/bias_ml_dh_utils
!pip install --upgrade tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import spacy
import numpy as np
import torch as tr
import matplotlib.pyplot as plt
import pickle
import copy

from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from collections import Counter
from spacy.tokenizer import Tokenizer


from bisect import bisect_left
from tqdm.notebook import tqdm

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

import bias_ml_dh_utils as utils

nlp = spacy.load("en_core_web_sm")
tokenizer = nlp.Defaults.create_tokenizer(nlp)

Here, you can choose between different data sets for the classification task and different Word Embeddings. 
For starters, we provide the standard glove 50 dimensinal embeddings provided by the authors of the glove paper. If you decide to do Session_1.3, you can load your custom word embeddings here.
You can choose between four different classification problems:

1. `yelp_sentiment_english`: Sentiment analysis on yelp reviews
2. `imdb_sentiment_english`: Sentiment analysis on imdb reviews
3. `amazon_sentiment_english`: Sentiment analysis on amazon reviews
4. `wikipedia`: Aggressive comments detection on talk pages on wikipedia



In [ ]:
data_identifier = "yelp_sentiment_english"
embedding_path = utils.download_dataset("glove.6B.50d")

#data_identifier = "wikipedia"
#embedding_path = "data/non_aggressive_comments_vec.txt"
#embedding_path = "data/aggressive_comments_vec.txt"
#embedding_path = "data/non_aggressive_comments_vec.txt"

In [ ]:
def load_data(data_identifier, vocab):
    
    sentiment_datasets = [
        "yelp_sentiment_english",
        "amazon_sentiment_english",
        "imdb_sentiment_english",
    ]
    
    if data_identifier in sentiment_datasets:
        
        path_to_data = utils.download_dataset(data_identifier)

        df = pd.read_csv(path_to_data, names=['document', 'label'], sep='\t')
        
        labels = df['label'].values
        doc_strings = df['document']

        
    elif data_identifier=="wikipedia":
        
        with open("data/wikipedia_toxic_classification_data", "rb") as fin:
            doc_strings, labels = pickle.load(fin)

        labels = np.array(labels)
        
    else:
        raise ValueError('data not known')

    documents = []
    for document in tqdm(tokenizer.pipe(doc_strings),desc="tokenize", total=len(doc_strings)):
        new_doc = []
        for t in document:
            try:
                new_doc.append(utils.index_sorted_list(vocab, t.text))
            except ValueError:
                pass
        
        documents.append(new_doc)
        
    return documents, labels

In [ ]:
def prepare_data(documents, labels, maxlen, pad_id):
    
    X = np.zeros((len(documents), max_len), dtype="int") + pad_id

    for idoc, doc in tqdm(enumerate(documents), desc="pad docs"):
        X[idoc,:len(doc)] = doc[:maxlen]
    
    X = tr.from_numpy(X)
    labels = tr.from_numpy(labels).float()
    
    x_train, x_test, y_train, y_test = train_test_split(
        X, labels,  
        test_size=0.2
    )

    x_val, x_test, y_val, y_test = train_test_split(
        x_test, y_test,  
        test_size=0.5
    )
    
    return x_train, x_val, x_test, y_train, y_val, y_test

In [ ]:
def get_batch(x_train, y_train, batch_size=50):
    '''get batch with random samples and equal number of samples from each class.
       resulting batch size is at most `batch_size`'''
    x_batches = []
    y_batches = []
    for cls in set(y_train.tolist()):
        x_batch = x_train[y_train == cls]
        y_batch = y_train[y_train == cls]
        perm = torch.randperm(x_batch.size(0))
        idx = perm[:batch_size//len(set(y_train.tolist()))]
        x_batch = x_batch[idx]
        y_batch = y_batch[idx]

        x_batches.append(x_batch)
        y_batches.append(y_batch)

    x_batch = torch.cat(x_batches)
    y_batch = torch.cat(y_batches)
    idx = torch.randperm(x_batch.size(0))
    x_batch = x_batch[idx]
    y_batch = y_batch[idx]

    return x_batch, y_batch

## Network Architecture

Here, you could change kernel sizes, number of filters, dropout rate etc.

In [ ]:
class Net(nn.Module):
    def __init__(self, embedding_matrix):
        super(Net, self).__init__()  
        filter_sizes = [3,4,5]
        num_filters = 100
        
        vocab_size, embedding_dim = embedding_matrix.shape
        
        #Embedding layer
        self.embedding_layer = nn.Embedding(embedding_matrix.shape[0], embedding_dim)
        self.embedding_layer.weight = nn.Parameter(tr.from_numpy(embedding_matrix).float())
        self.embedding_layer.weight.requires_grad = False
        
        #Convolution layer
        
        self.convolution_layer = nn.ModuleList([nn.Conv2d(1, num_filters, (K, embedding_dim)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(len(self.convolution_layer)*num_filters, 1)
        
    def forward(self,x):
        x = self.embedding_layer(x)
        x = x.unsqueeze(1)  
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convolution_layer] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  
        x = tr.cat(x, 1)
        x = self.dropout(x)
        logit = self.linear(x)
        return(logit.view(-1))

## Training Loop

Here, you could change learning rate (`lr`) and batch size `get_batch(x_train, y_train, batch_size=YOUR_BATCH_SIZE)`

In [ ]:
def train_classifier(net, x_train, y_train, x_val, y_val, max_it=200):
    bestmodel = copy.deepcopy(net)
    #sets optimizer and loss function
    optimizer = optim.Adam(net.parameters(), lr=0.002)
    criterion = nn.BCEWithLogitsLoss()

    loss_hist = []
    loss_val_hist = []
    
    for it in tqdm(range(max_it)):
        
        x_batch, y_batch = get_batch(x_train, y_train)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        loss_hist.append(loss.item())
        
        every_tenth_iteration = (it%10) == 0
        last_iteration = it == (max_it-1)
        if every_tenth_iteration or last_iteration:

            outputs_val = net.forward(x_val)
            val_loss = criterion(outputs_val, y_val)
            if len(loss_val_hist) > 0 and val_loss < min(loss_val_hist):
                bestmodel = copy.deepcopy(net)
            loss_val_hist.append(val_loss.item())

            print(
                "training loss: {:0.3f}".format(loss_hist[-1]),
                "validati loss: {:0.3f}".format(loss_val_hist[-1]),
            )
            
    y_predict = (outputs_val.detach().numpy()>=0).astype(int).ravel()
    print(classification_report(y_val, y_predict))

    plt.plot(loss_hist)
    plt.plot(np.arange(0,max_it+1,10),loss_val_hist)
    plt.legend(['training_loss', 'validation_loss'])
    plt.savefig('loss.png', dpi=300)
    
    outputs_val = net.forward(x_test)
    y_predict = (outputs_val.detach().numpy()>=0).astype(int).ravel()
    print(classification_report(y_test, y_predict))
    return bestmodel

In [ ]:
def predict_label(input_data, max_len, vocab, pad_id, label_names = ['negative','positive']):

    
    X = torch.zeros((1,max_len)).long() + pad_id
    for it, t in enumerate(nlp(input_data)):
        try:
            X[:,it] = utils.index_sorted_list(vocab, t.text)
        except ValueError:
            print(f"`{t.text}` not found in vocab")

    net.eval()
    output = net.forward(X)
    label = tr.clamp(tr.sign(output.detach()),0,1)

    print("The predicted label is: ",label_names[int(label)])
    print(output)
    
    return output.detach()

## Entry points for the pipeline

In [ ]:
# load embedding matrix
embedding_matrix, vocab = utils.create_embedding_matrix(embedding_path)

In [ ]:
# load data
documents, labels = load_data(data_identifier, vocab)

### Let's look at some samples..

In [ ]:
def decode(sample, vocab):
    decoded = []
    for token in sample:
        decoded.append(vocab[token])
    return " ".join(decoded)

random_doc_id = np.random.randint(0,len(documents))
print(decode(documents[random_doc_id], vocab), labels[random_doc_id])

Here, you can change the maximum sequence length (`max_len`)

In [ ]:
max_len = 100
pad_id = utils.index_sorted_list(vocab, "[PAD]")

x_train, x_val, x_test, y_train, y_val, y_test = prepare_data(documents, labels, max_len, pad_id)

Here, you can change the number of training steps

In [ ]:
net = Net(embedding_matrix)

num_training_steps = 100
net = train_classifier(net, x_train, y_train, x_val, y_val, num_training_steps)

## Evaluate resulting model

Try out new data! Test your model on biases by classifying sentences that you think include biases.


In [ ]:
input_sentence = 'this movie was really good'
print(data)
_ = predict_label(input_sentence,  max_len, vocab, pad_id)

In [ ]:
# For the wikipedia corpus, you may use these labels:
input_sentence = "we like you"
_ = predict_label(input_sentence,  max_len, vocab, pad_id, ["non-aggressive", "aggressive"])